In [2]:
import pandas as pd
import os
import datetime
from datetime import timedelta
from dateutil.relativedelta import relativedelta
import plotly.express as px
import plotly
pd.options.display.float_format = '{:,.2f}'.format

In [3]:
print('Inicializado Rotina json_collect.py.')
os.chdir(sys.path[0])
# os.chdir(os.path.join(os.getcwd(),"json_collect.py"))                  # Modifica o diretório de trabalho, para os caminhos relativos funcionar. (https://stackoverflow.com/questions/1432924/python-change-the-scripts-working-directory-to-the-scripts-own-directory)
# print(os.getcwd())
%run ./json_collect.py                      

Inicializado Rotina json_collect.py.


In [4]:
# sites: https://dadosabertos.bcb.gov.br/dataset/expectativas-mercado
# Sintase: https://dadosabertos.bcb.gov.br/dataset/expectativas-mercado/resource/d420a704-75a7-4f45-8f4b-0fca813c70f0
# documentação: https://olinda.bcb.gov.br/olinda/servico/Expectativas/versao/v1/documentacao

In [5]:
# Relações datas das reunições inseridas manualmente quando não tem previsão
df_selic_data=pd.read_csv('./input/selic_datas.csv',index_col='Reuniao')
# Dados baixados pelo .json
df_selic=pd.read_csv('./output/selic_data_df.csv',index_col='Reuniao')
df_selic=df_selic.drop(columns=['Unnamed: 0','Indicador'])
df_selic['Data']=df_selic.apply(lambda x: datetime.datetime.strptime(x.Data,'%Y-%m-%d').date(),axis=1)

# Relaciona a reunião com sua Data.
df_selic['data_reuniao']=''
for i in range(0, len(df_selic)):
    busca_data=''
    try:
        busca_data=df_selic_data.loc[df_selic.index[i],'data_reuniao']
        busca_data=datetime.datetime.strptime(busca_data,'%Y-%m-%d').date()
        df_selic.loc[df_selic.index[i],'data_reuniao']=busca_data

    except:
        #### será definida um expectativa para a data.
        # busca_data=df_selic_data.loc[df_selic.index[i],'data_reuniao']
        # df_selic.loc[df_selic.index[i],'data_reuniao']=busca_data
        continue

# df_selic['data_reuniao']=df_selic.apply(lambda x: datetime.datetime.strptime(x.data_reuniao,'%Y-%m-%d').date(),axis=1)
# df_selic=df_selic[df_selic['data_reuniao']!='']
df_selic=df_selic.sort_values(by=['Data','data_reuniao','baseCalculo'],ascending=[False,True,False])

'''Base de cálculo para as estatísticas
 baseada no prazo de validade das expectativas
 informadas pelas instituições informantes:
  - 0: uso das expectativas mais recentes informadas
   pelas instituições participantes a partir do
   30º dia anterior à data de cálculo das estatísticas 
  - 1: uso das expectativas mais recentes informadas
   pelas instituições participantes a partir do
    4º dia útil anterior à data de cálculo das estatísticas'''

df_selic.head(20)

,Data,Media,Mediana,DesvioPadrao,Minimo,Maximo,numeroRespondentes,baseCalculo,data_reuniao
Reuniao,,,,,,,,,
R6/2022,2022-08-12,13.78,13.75,0.08,13.75,14.00,69,1,2022-09-21
R6/2022,2022-08-12,13.82,13.75,0.14,13.50,14.25,130,0,2022-09-21
R7/2022,2022-08-12,13.79,13.75,0.10,13.75,14.25,69,1,2022-10-26
R7/2022,2022-08-12,13.83,13.75,0.15,13.50,14.25,130,0,2022-10-26
R8/2022,2022-08-12,13.78,13.75,0.10,13.50,14.25,69,1,2022-12-07
R8/2022,2022-08-12,13.83,13.75,0.15,13.50,14.25,130,0,2022-12-07
R1/2023,2022-08-12,13.77,13.75,0.12,13.25,14.25,67,1,2023-02-01
R1/2023,2022-08-12,13.80,13.75,0.18,13.25,14.25,124,0,2023-02-01
R2/2023,2022-08-12,13.71,13.75,0.23,12.75,14.25,67,1,2023-03-22


In [6]:
# Gráfico iterativo utilizando o plotly
fig=px.line(df_selic,x='data_reuniao',y='Mediana', color='Data')
# Gerando arquivo com o resultado
# plotly.offline.plot(fig, show_link = True,filename=f'./output/IPCA_forcast.html')
fig.show()

In [7]:
# Informada à partir do 4º dia útil anterior à data de cálculo da expectativa.
df_selic=df_selic[df_selic['baseCalculo']==1]           
# Expectativa mais atual
df_selic_ultima=df_selic[df_selic['Data']==df_selic['Data'].max()]
# df_selic_ultima['mediana_mensal']=df_selic_ultima.apply(lambda x: (1+(x.Mediana/100))**(1/12)-1,axis=1)

In [8]:
# Insere coluna com números de dias no período (entre duas datas)
df_selic_ultima.loc[df_selic_ultima.index[0],'periodo']=(df_selic_ultima.loc[df_selic_ultima.index[0],'data_reuniao']-datetime.datetime.today().date()).days
# Define número índice de ref. 100 na data de hoje.
selic_atual=''
while type(selic_atual)!=float:
    try:
        selic_atual=float(input('Qual é a SELIC atual?(\d\d.d\d\)'))
    except:
        continue    
df_selic_ultima.loc[df_selic_ultima.index[0],'num_indice']=100*(1+selic_atual/100)**(df_selic_ultima.loc[df_selic_ultima.index[0],'periodo']/365)
for i in range(1,len(df_selic_ultima)):
    df_selic_ultima.loc[df_selic_ultima.index[i],'periodo']=(df_selic_ultima.loc[df_selic_ultima.index[i],'data_reuniao']-df_selic_ultima.loc[df_selic_ultima.index[i-1],'data_reuniao']).days
    df_selic_ultima.loc[df_selic_ultima.index[i],'num_indice']=df_selic_ultima.loc[df_selic_ultima.index[i-1],'num_indice']*(1+df_selic_ultima.loc[df_selic_ultima.index[i-1],'Mediana']/100)**(df_selic_ultima.loc[df_selic_ultima.index[i],'periodo']/365)
df_selic_ultima.to_csv('./output/selic_ultima_expectativa.csv',float_format='%.2f')


C:\Users\bruno.bfn\AppData\Local\Temp\ipykernel_1336\1363766147.py:2: SettingWithCopyWarning:


A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy

C:\Users\bruno.bfn\AppData\Local\Temp\ipykernel_1336\1363766147.py:10: SettingWithCopyWarning:


A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy



In [9]:
df_selic_ultima.tail()

,Data,Media,Mediana,DesvioPadrao,Minimo,Maximo,numeroRespondentes,baseCalculo,data_reuniao,periodo,num_indice
Reuniao,,,,,,,,,,,
R1/2024,2022-08-12,10.41,10.50,0.84,8.50,12.00,60,1,2024-01-31,49.00,119.53
R2/2024,2022-08-12,9.91,10.00,0.91,7.50,11.50,59,1,2024-03-20,49.00,121.15
R3/2024,2022-08-12,9.47,9.50,0.97,7.50,11.50,59,1,2024-05-08,49.00,122.71
R4/2024,2022-08-12,9.10,9.00,0.98,7.00,11.25,58,1,2024-06-26,49.00,124.21
R5/2024,2022-08-12,8.86,8.75,0.95,7.00,11.00,42,1,2024-08-14,49.00,125.66


In [10]:
#### Função que calcula a SELIC do período entre data atual e data final
def calc_selic_periodo(df,data_f):
    # DF é um dataframe que contenha a culuna com num_indice
    
    # Utiliza a taxa digitada real vigente
    if data_f<=df.loc[df.index[0],'data_reuniao']:
        n_dias=(data_f-datetime.datetime.today().date()).days
        return 100*(1+selic_atual/100)**(n_dias/365)
    
    # Utiliza o número índice anterior + variação dos dias dentro do período
    else:
        # Último número índice
        df=df[df['data_reuniao']<=data_f]
        n_indice=df.loc[df.index[-1],'num_indice']
        taxa=df.loc[df.index[-1],'Mediana']
        n_dias=(data_f-df.loc[df.index[-1],'data_reuniao']).days
        return n_indice*(1+taxa/100)**(n_dias/365)


In [11]:
calc_selic_periodo(df_selic_ultima,datetime.datetime(2022,10,25).date())

102.35692736882415

In [12]:
df_monthly=pd.read_csv('./output/monthly_data_df.csv')
df_monthly=df_monthly.drop(columns=['Unnamed: 0'])

# Parses
df_monthly['Data']=df_monthly.apply(lambda x: datetime.datetime.strptime(x.Data,'%Y-%m-%d').date(),axis=1)

df_monthly['DataReferencia']=df_monthly.apply(lambda x: datetime.datetime.strptime(x.DataReferencia,'%m/%Y').date(),axis=1)
delta = relativedelta(months=1)
delta_d = timedelta(days=1)
df_monthly['DataReferencia']=df_monthly.apply(lambda x: x.DataReferencia+delta-delta_d,axis=1)

df_monthly=df_monthly.sort_values(by=['Data','DataReferencia','numeroRespondentes'],ascending=[False,True,False])
# df_monthly=df_selic[df_selic['baseCalculo']==1]           # não sei qual é a diferença da base de cálculo.
df_monthly.head(20)

,Indicador,Data,DataReferencia,Media,Mediana,Minimo,Maximo,numeroRespondentes
419,Taxa de desocupação,2022-08-12,2022-07-31,9.22,9.20,8.63,10.10,51
418,Taxa de desocupação,2022-08-12,2022-07-31,9.19,9.10,8.80,10.10,17
403,IPCA,2022-08-12,2022-08-31,-0.16,-0.19,-0.58,0.55,141
405,Câmbio,2022-08-12,2022-08-31,5.25,5.23,4.91,5.68,109
417,IPCA Administrados,2022-08-12,2022-08-31,-1.50,-1.64,-2.76,1.60,80
401,IGP-M,2022-08-12,2022-08-31,0.06,0.10,-0.90,0.85,78
409,IPCA Livres,2022-08-12,2022-08-31,0.35,0.38,-0.14,0.64,74
402,IPCA,2022-08-12,2022-08-31,-0.21,-0.20,-0.55,0.40,68
415,IPCA Alimentação no domicílio,2022-08-12,2022-08-31,0.53,0.50,-0.25,1.40,67
411,IPCA Serviços,2022-08-12,2022-08-31,0.26,0.30,-0.65,0.61,66


In [13]:
df_monthly['Indicador'].unique()

array(['Taxa de desocupação', 'IPCA', 'Câmbio', 'IPCA Administrados',
       'IGP-M', 'IPCA Livres', 'IPCA Alimentação no domicílio',
       'IPCA Serviços', 'IPCA Bens industrializados'], dtype=object)

In [14]:
# Gráfico iterativo utilizando o plotly
fig=px.line(df_monthly[df_monthly['Indicador']=='IPCA'],x='DataReferencia',y='Mediana', color='Data')
# Gerando arquivo com o resultado
plotly.offline.plot(fig, show_link = True,filename=f'./output/IPCA_forcast.html')
# fig.show()

# Gráfico iterativo utilizando o plotly
fig=px.line(df_monthly[df_monthly['Indicador']=='IGP-M'],x='DataReferencia',y='Mediana', color='Data')
# Gerando arquivo com o resultado
plotly.offline.plot(fig, show_link = True,filename=f'./output/IGPM_forcast.html')
# fig.show()

# Gráfico iterativo utilizando o plotly
fig=px.line(df_monthly[df_monthly['Indicador']=='Câmbio'],x='DataReferencia',y='Mediana', color='Data')
# Gerando arquivo com o resultado
plotly.offline.plot(fig, show_link = True,filename=f'./output/cambio_forcast.html')
# fig.show()

# Gráfico iterativo utilizando o plotly
fig=px.line(df_monthly[df_monthly['Indicador']=='Taxa de desocupação'],x='DataReferencia',y='Mediana', color='Data')
# Gerando arquivo com o resultado
plotly.offline.plot(fig, show_link = True,filename=f'./output/tx_desocupacao_forcast.html')
# fig.show()

'./output/tx_desocupacao_forcast.html'

In [15]:
# Expectativa mais atual para IPCA
df_ipca=df_monthly[df_monthly['Indicador']=='IPCA']

data_previsao=df_ipca['Data'].max()
df_ipca_ultima=df_ipca[df_ipca['Data']==data_previsao]
df_ipca_ultima=df_ipca_ultima.groupby(['DataReferencia']).mean() # adotado média 

df_ipca_ultima['cumsum_mediana'] = df_ipca_ultima['Mediana'].cumsum()
#cumsum()-> soma acumulada de uma lista ou array termo a termo
#groupby -> delimita em grupos separados para análise
# df['Adjusted Quantity'] = df.groupby('ação_ref')['Adjusted Quantity'].cumsum()

In [16]:
# Insere coluna com números de dias no período (entre duas datas)
df_ipca_ultima.loc[df_ipca_ultima.index[0],'periodo']=(df_ipca_ultima.index[0]-datetime.datetime.today().date()).days

# Define número índice de ref. 100 na data de hoje.
df_ipca_ultima.loc[df_ipca_ultima.index[0],'num_indice']=100*(1+df_ipca_ultima.loc[df_ipca_ultima.index[0],'Mediana']/100)**(df_ipca_ultima.loc[df_ipca_ultima.index[0],'periodo']/30)
for i in range(1,len(df_ipca_ultima)):
    df_ipca_ultima.loc[df_ipca_ultima.index[i],'periodo']=(df_ipca_ultima.index[i]-df_ipca_ultima.index[i-1]).days
    df_ipca_ultima.loc[df_ipca_ultima.index[i],'num_indice']=df_ipca_ultima.loc[df_ipca_ultima.index[i-1],'num_indice']*(1+df_ipca_ultima.loc[df_ipca_ultima.index[i],'Mediana']/100)**(df_ipca_ultima.loc[df_ipca_ultima.index[i],'periodo']/30)
df_ipca_ultima.to_csv('./output/selic_ultima_expectativa.csv')
df_ipca_ultima

,Media,Mediana,Minimo,Maximo,numeroRespondentes,cumsum_mediana,periodo,num_indice
DataReferencia,,,,,,,,
2022-08-31,-0.19,-0.20,-0.56,0.48,104.50,-0.20,11.00,99.93
2022-09-30,0.46,0.46,0.14,0.75,104.50,0.27,30.00,100.39
2022-10-31,0.54,0.54,0.20,0.84,103.00,0.81,31.00,100.95
2022-11-30,0.53,0.53,0.13,0.76,102.00,1.33,30.00,101.48
2022-12-31,0.74,0.74,0.23,1.10,102.00,2.07,31.00,102.26
2023-01-31,0.77,0.73,0.29,1.50,100.00,2.80,31.00,103.03
2023-02-28,0.67,0.67,0.30,1.15,100.00,3.47,28.00,103.67
2023-03-31,0.47,0.47,0.20,1.06,100.00,3.93,31.00,104.17
2023-04-30,0.45,0.45,0.17,0.75,100.00,4.38,30.00,104.63


In [18]:
# from matplotlib.pyplot import figure
# figure(figsize=(10, 5), dpi=120)
# plt.plot(df_ipca_ultima.index, df_ipca_ultima['Mediana'], label=f'ipca_expectaiva_{data_previsao}')
# plt.xticks(rotation=90)
# plt.legend()
# plt.show()


In [19]:
#### Função que calcula a IPCA do período entre data hoje e data final
def calc_ipca_periodo(df,data_f):
    # DF é um dataframe que contenha a coluna com num_indice
    
    # Utiliza a taxa digitada real vigente
    if data_f<=df.index[0]:
        # print('primeiro mês')
        n_dias=(data_f-datetime.datetime.today().date()).days
        # print(n_dias)
        return 100*(1+df.loc[df.index[0],'Mediana']/100)**(n_dias/30)
    
    # Utiliza o número índice anterior + variação dos dias dentro do período
    else:
        # Último número índice
        df_anterior=df[df.index<=data_f]
        # print(df_anterior)
        n_indice=df_anterior.loc[df_anterior.index[-1],'num_indice']
        # print(f'n_indice:{n_indice}')
        #busca o 1º termo depois do corte da df_anterior
        taxa=df.loc[df.index[len(df_anterior)],'Mediana']    
        # print(f'taxa:{taxa}')
        n_dias=(data_f-df_anterior.index[-1]).days
        # print(f'n_dias:{n_dias}')
        return n_indice*(1+taxa/100)**(n_dias/30)

In [20]:
calc_ipca_periodo(df_ipca_ultima,datetime.datetime(2022,11,30).date())

101.4804141609918

In [21]:
df_ipca_ultima

,Media,Mediana,Minimo,Maximo,numeroRespondentes,cumsum_mediana,periodo,num_indice
DataReferencia,,,,,,,,
2022-08-31,-0.19,-0.20,-0.56,0.48,104.50,-0.20,11.00,99.93
2022-09-30,0.46,0.46,0.14,0.75,104.50,0.27,30.00,100.39
2022-10-31,0.54,0.54,0.20,0.84,103.00,0.81,31.00,100.95
2022-11-30,0.53,0.53,0.13,0.76,102.00,1.33,30.00,101.48
2022-12-31,0.74,0.74,0.23,1.10,102.00,2.07,31.00,102.26
2023-01-31,0.77,0.73,0.29,1.50,100.00,2.80,31.00,103.03
2023-02-28,0.67,0.67,0.30,1.15,100.00,3.47,28.00,103.67
2023-03-31,0.47,0.47,0.20,1.06,100.00,3.93,31.00,104.17
2023-04-30,0.45,0.45,0.17,0.75,100.00,4.38,30.00,104.63


In [22]:
# Colocar o índice para coluna
# df_ipca_from_now.reset_index() 


In [23]:
# Gráfico iterativo utilizando o plotly
fig=px.line(df_ipca_ultima,x=df_ipca_ultima.index,y=['cumsum_mediana'])
# Gerando arquivo com o resultado
# plotly.offline.plot(fig, show_link = True,filename=f'./output/tx_desocupacao_forcast.html')
fig.show()

In [24]:
# Relação de datas até o mínimo entre os dataframes, para evitar erros
lista_datas_todos_dias=pd.date_range(start=datetime.datetime.today().date(),end=min(df_selic_ultima['data_reuniao'].max(),df_ipca_ultima.index.max()))
df_juros_real=pd.DataFrame(index=lista_datas_todos_dias,columns=['num_indice_selic','num_indice_ipca'],dtype='float64')
for i in range(0,len(df_juros_real)):
    df_juros_real.loc[df_juros_real.index[i],'num_indice_selic']=calc_selic_periodo(df_selic_ultima,df_juros_real.index[i].date())
    df_juros_real.loc[df_juros_real.index[i],'num_indice_ipca']=calc_ipca_periodo(df_ipca_ultima,df_juros_real.index[i].date())
df_juros_real

,num_indice_selic,num_indice_ipca
2022-08-20,100.00,100.00
2022-08-21,100.04,99.99
2022-08-22,100.07,99.99
2022-08-23,100.11,99.98
2022-08-24,100.14,99.97
...,...,...
2024-08-10,125.54,110.02
2024-08-11,125.57,110.03
2024-08-12,125.60,110.04
2024-08-13,125.63,110.04


In [25]:
# Não utilizada
def area(tipo,date):
    #tipo={selic,ipca}
    area=0
    df=df_juros_real[df_juros_real.index<=date]
    if (tipo=='selic' or tipo=='ipca'):
        for i in range(0,len(df)):
            area=area+1*df.loc[df.index[i],f'num_indice_{tipo}']
                
        return area
    else:
        print('Erro na escolha do tipo (selic ou ipca)')
    return 0

In [26]:
# df_juros_real['area_selic']=0
# for i in range(0,len(df_juros_real)):
#     # print(area('selic',df_juros_real.index[i]))
#     df_juros_real.loc[df_juros_real.index[i], 'area_selic']=area('selic',df_juros_real.index[i])
#     df_juros_real.loc[df_juros_real.index[i], 'area_ipca']=area('ipca',df_juros_real.index[i])

# df_juros_real

In [27]:
df_juros_real['dif_num_indice']=df_juros_real.apply(lambda x: x.num_indice_selic-x.num_indice_ipca+100,axis=1)
# df_juros_real['dif_area']=df_juros_real.apply(lambda x: x.area_selic-x.area_ipca,axis=1)
# df_juros_real['dif_area_por_dia']=df_juros_real.apply(lambda x: x.dif_area/(x.index-))
df_juros_real.to_csv('./output/num_indice_selic_ipca_diferenca.csv',float_format='%.2f')
df_juros_real


,num_indice_selic,num_indice_ipca,dif_num_indice
2022-08-20,100.00,100.00,100.00
2022-08-21,100.04,99.99,100.04
2022-08-22,100.07,99.99,100.08
2022-08-23,100.11,99.98,100.13
2022-08-24,100.14,99.97,100.17
...,...,...,...
2024-08-10,125.54,110.02,115.51
2024-08-11,125.57,110.03,115.54
2024-08-12,125.60,110.04,115.56
2024-08-13,125.63,110.04,115.59


In [28]:
# Gráfico iterativo utilizando o plotly
fig=px.line(df_juros_real,x=df_juros_real.index,y=['num_indice_selic','num_indice_ipca','dif_num_indice'])
# Gerando arquivo com o resultado
# plotly.offline.plot(fig, show_link = True,filename=f'./output/IPCA_forcast.html')
fig.show()

In [29]:
from time import strptime
from pyxirr import xirr
# calculo da IRR
def cal_irr(x,date):                #DF refer^rencia e a data de análise
    if date==x.index[0]:
        return (0,0,0,0)
    dates=[x.index[0],date]
    amounts=[-x.loc[x.index[0],'dif_num_indice'],x.loc[date,'dif_num_indice']]
    
    # date=datetime.datetime.strptime(date,'%Y-%m-%d').date()
    # x=x[x.index==x.index[0] | x.index.date()==date]
    # dates=x.index
    # amounts=x['dif_num_indice']
    t_aa=xirr(dates, amounts)
    t_am=(1+t_aa)**(1/12)-1
    # print('TII a.a.(%): ',t_aa*100)
    # print('TII a.m.(%): ',t_am*100)
    # print('Soma dos valores: ',-x['Valor'].sum())
    # print(dates)
    # print(amounts)
    return (t_aa*100,t_am*100,dates,amounts)

In [30]:
cal_irr(df_juros_real,datetime.datetime(2023,8,20))[0]

7.839352856851605

In [31]:
for i in range(0,len(df_juros_real)):
    df_juros_real.loc[df_juros_real.index[i],'juros_real_acum_from_now']=cal_irr(df_juros_real,df_juros_real.index[i])[0]
    df_juros_real.loc[df_juros_real.index[i],'juros_real_am']=cal_irr(df_juros_real,df_juros_real.index[i])[1]

df_juros_real.head(30)

,num_indice_selic,num_indice_ipca,dif_num_indice,juros_real_acum_from_now,juros_real_am
2022-08-20,100.00,100.00,100.00,0.00,0.00
2022-08-21,100.04,99.99,100.04,16.51,1.28
2022-08-22,100.07,99.99,100.08,16.51,1.28
2022-08-23,100.11,99.98,100.13,16.51,1.28
2022-08-24,100.14,99.97,100.17,16.51,1.28
2022-08-25,100.18,99.97,100.21,16.51,1.28
2022-08-26,100.21,99.96,100.25,16.51,1.28
2022-08-27,100.25,99.95,100.29,16.50,1.28
2022-08-28,100.28,99.95,100.34,16.50,1.28
2022-08-29,100.32,99.94,100.38,16.50,1.28


In [32]:
for i in range(0,len(df_juros_real)):
    # Primeiro dia do mês
    inicio_mes=df_juros_real.index[i]-timedelta(df_juros_real.index[i].day-1)
    if inicio_mes<=df_juros_real.index[0]:
        inicio_mes=df_juros_real.index[0]
    # print(f'Início do mês: {inicio_mes} - até: {df_juros_real.index[i]}')
    # Slice do DF do primeiro dia do mês até o dia da análise
    try:
        df_juros_real_i=df_juros_real[df_juros_real.index>=inicio_mes]
        df_juros_real_i=df_juros_real_i[df_juros_real_i.index<=df_juros_real.index[i]]
        # print(df_juros_real_i)
        df_juros_real.loc[df_juros_real.index[i],'juros_real_mesal']=cal_irr(df_juros_real_i,df_juros_real.index[i])[0]
    except:
        continue
df_juros_real.head(30)

,num_indice_selic,num_indice_ipca,dif_num_indice,juros_real_acum_from_now,juros_real_am,juros_real_mesal
2022-08-20,100.00,100.00,100.00,0.00,0.00,0.00
2022-08-21,100.04,99.99,100.04,16.51,1.28,16.51
2022-08-22,100.07,99.99,100.08,16.51,1.28,16.51
2022-08-23,100.11,99.98,100.13,16.51,1.28,16.51
2022-08-24,100.14,99.97,100.17,16.51,1.28,16.51
2022-08-25,100.18,99.97,100.21,16.51,1.28,16.51
2022-08-26,100.21,99.96,100.25,16.51,1.28,16.51
2022-08-27,100.25,99.95,100.29,16.50,1.28,16.50
2022-08-28,100.28,99.95,100.34,16.50,1.28,16.50
2022-08-29,100.32,99.94,100.38,16.50,1.28,16.50


In [33]:
# Gráfico iterativo utilizando o plotly
fig=px.line(df_juros_real,x=df_juros_real.index,y=['juros_real_acum_from_now','juros_real_mesal'])
# Gerando arquivo com o resultado
plotly.offline.plot(fig, show_link = True,filename=f'./output/Juros_real_forcast.html')
fig.show()
